In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from scripts.cv_utils import *

In [19]:
class SEBlock(nn.Module):
    def __init__(self, channels, reduction):
        super(SEBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)

    def forward(self, x):
        B, C, W, H = x.shape
        out = self.global_avg_pool(x)
        out = torch.flatten(out, 1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.sigmoid(out)
        out = out.view(B, C, 1, 1)
        return x * out

In [20]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding='same')
        self.bn1 = nn.BatchNorm2d(64)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        return x

In [21]:
x = torch.randn(1, 3, 64, 64)
model = SimpleNet()
x = model(x)
print(x.shape)

se_block = SEBlock(64, 16)
x = se_block(x)
print(x.shape)

torch.Size([1, 64, 64, 64])
torch.Size([1, 64, 64, 64])


In [9]:
64 // 16

4